# 🚀 MSSQL Text-to-SQL with Multi-Provider LLM Integration

## Overview

This notebook provides a comprehensive text-to-SQL solution that:

1. **Connects to MSSQL Database** - Establishes secure connection to SQL Server using pymssql
2. **Extracts Database Metadata** - Retrieves table/column information and saves to Excel
3. **Multi-Provider LLM Support** - OpenAI, Anthropic, Google Gemini, DeepSeek for natural language processing
4. **Direct SQL Execution** - Runs generated queries directly on the MSSQL database
5. **Interactive Interface** - User-friendly widgets for query input and execution

### 🔧 Prerequisites

Before running this notebook, ensure you have:

- **MSSQL Server** access with proper credentials
- **API Keys** for at least one LLM provider
- **Python packages**: pandas, sqlalchemy, pymssql, openpyxl, ipywidgets, requests, python-dotenv

### 🔑 Environment Variables

Set up your API keys and database connection:

```env
# Database Connection
MSSQL_SERVER=your_server_name
MSSQL_DATABASE=your_database_name
MSSQL_USERNAME=your_username
MSSQL_PASSWORD=your_password
MSSQL_PORT=1433

# LLM API Keys (at least one required)
OPENAI_API_KEY=your_openai_key
ANTHROPIC_API_KEY=your_anthropic_key
GOOGLE_API_KEY=your_google_key
DEEPSEEK_API_KEY=your_deepseek_key
```

## 1. Import Required Libraries

In [1]:
# Core libraries for data manipulation and database connectivity
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Database connectivity
import sqlalchemy
from sqlalchemy import create_engine, text, MetaData, inspect
import pymssql
import urllib.parse

# Excel file handling
import openpyxl
from openpyxl import Workbook

# Environment variables and configuration
from dotenv import load_dotenv

# LLM API calls
import requests
import json

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# Set up plotting styles
plt.style.use('default')
sns.set_palette("husl")
pyo.init_notebook_mode(connected=True)

# Load environment variables from .env file
load_dotenv()

print("✅ All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🗄️ SQLAlchemy version: {sqlalchemy.__version__}")
print(f"🔌 pymssql version: {pymssql.__version__}")

# Check for required environment variables
required_env_vars = ['MSSQL_SERVER', 'MSSQL_DATABASE']
missing_vars = [var for var in required_env_vars if not os.getenv(var)]

if missing_vars:
    print(f"\n⚠️  Missing required environment variables: {missing_vars}")
    print("Please set these in your .env file or environment")
else:
    print("✅ Required database environment variables found")

# Check for LLM API keys
llm_providers = {
    'OpenAI': 'OPENAI_API_KEY',
    'Anthropic': 'ANTHROPIC_API_KEY', 
    'Google': 'GOOGLE_API_KEY',
    'DeepSeek': 'DEEPSEEK_API_KEY'
}

available_providers = []
for provider, env_var in llm_providers.items():
    if os.getenv(env_var):
        available_providers.append(provider)
        print(f"✅ {provider} API key found")
    else:
        print(f"❌ {provider} API key missing ({env_var})")

if not available_providers:
    print("\n⚠️  No LLM API keys found! Please set at least one API key.")
else:
    print(f"\n🤖 Available LLM providers: {', '.join(available_providers)}")

✅ All libraries imported successfully!
📊 Pandas version: 2.3.1
🗄️ SQLAlchemy version: 2.0.42
🔌 pymssql version: 2.3.8
✅ Required database environment variables found
✅ OpenAI API key found
✅ Anthropic API key found
✅ Google API key found
✅ DeepSeek API key found

🤖 Available LLM providers: OpenAI, Anthropic, Google, DeepSeek


## 2. Connect to MSSQL Database

This section establishes a connection to the MSSQL Server database using SQLAlchemy with the pymssql driver.

In [ ]:
class MSSQLConnector:
    """
    MSSQL Database Connector using SQLAlchemy and pymssql
    """
    
    def __init__(self):
        self.server = os.getenv('MSSQL_SERVER')
        self.database = os.getenv('MSSQL_DATABASE')
        self.username = os.getenv('MSSQL_USERNAME')
        self.password = os.getenv('MSSQL_PASSWORD')
        self.port = int(os.getenv('MSSQL_PORT', os.getenv('MSSQL_PORT')))
        self.use_windows_auth = os.getenv('MSSQL_USE_WINDOWS_AUTH', 'false').lower() == 'true'
        
        self.engine = None
        self.connection_string = None
        
    def create_connection_string(self):
        """Create the connection string for MSSQL using pymssql"""
        if self.use_windows_auth:
            # Windows Authentication - pymssql doesn't support this directly
            # You would need to use SSPI/Kerberos which is complex
            raise ValueError(
                "Windows Authentication is not supported with pymssql. "
                "Please use SQL Server authentication with username and password."
            )
        else:
            # SQL Server Authentication
            if not self.username or not self.password:
                raise ValueError("Username and password required for SQL Server authentication")
            
            # URL encode the password to handle special characters
            encoded_password = urllib.parse.quote_plus(self.password)
            encoded_username = urllib.parse.quote_plus(self.username)
            
            # pymssql connection string format
            self.connection_string = (
                f"mssql+pymssql://{encoded_username}:{encoded_password}@"
                f"{self.server}:{self.port}/{self.database}"
            )
        
        return self.connection_string
    
    def connect(self):
        """Establish database connection"""
        try:
            if not self.connection_string:
                self.create_connection_string()
            
            self.engine = create_engine(self.connection_string)
            
            # Test the connection
            with self.engine.connect() as conn:
                result = conn.execute(text("SELECT 1 as test")).fetchone()
                if result[0] == 1:
                    print("✅ Successfully connected to MSSQL database!")
                    print(f"📊 Server: {self.server}:{self.port}")
                    print(f"🗄️ Database: {self.database}")
                    return True
        
        except Exception as e:
            print(f"❌ Failed to connect to database: {str(e)}")
            print("\n💡 Troubleshooting tips:")
            print("1. Check if SQL Server is running")
            print("2. Verify server name and database name")
            print("3. Check credentials (username and password)")
            print("4. Ensure SQL Server is configured to allow TCP/IP connections")
            print("5. Check firewall settings and network connectivity")
            print("6. Verify the port number (default: 1433)")
            return False
    
    def execute_query(self, query, params=None):
        """Execute a SQL query and return results as DataFrame"""
        try:
            if not self.engine:
                print("❌ No database connection. Call connect() first.")
                return None
            
            if params:
                df = pd.read_sql_query(text(query), self.engine, params=params)
            else:
                df = pd.read_sql_query(text(query), self.engine)
            
            return df
        
        except Exception as e:
            print(f"❌ Error executing query: {str(e)}")
            return None
    
    def get_tables(self):
        """Get list of all tables in the database"""
        query = """
        SELECT 
            TABLE_SCHEMA,
            TABLE_NAME,
            TABLE_TYPE
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_TYPE = 'BASE TABLE'
        ORDER BY TABLE_SCHEMA, TABLE_NAME
        """
        return self.execute_query(query)
    
    def get_columns(self, schema_name=None, table_name=None):
        """Get column information for tables"""
        query = """
        SELECT 
            TABLE_SCHEMA,
            TABLE_NAME,
            COLUMN_NAME,
            DATA_TYPE,
            CHARACTER_MAXIMUM_LENGTH,
            IS_NULLABLE,
            COLUMN_DEFAULT,
            ORDINAL_POSITION
        FROM INFORMATION_SCHEMA.COLUMNS
        """
        
        conditions = []
        params = {}
        
        if schema_name:
            conditions.append("TABLE_SCHEMA = :schema_name")
            params['schema_name'] = schema_name
            
        if table_name:
            conditions.append("TABLE_NAME = :table_name")
            params['table_name'] = table_name
        
        if conditions:
            query += " WHERE " + " AND ".join(conditions)
        
        query += " ORDER BY TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION"
        
        return self.execute_query(query, params if params else None)

# Initialize the database connector
db_connector = MSSQLConnector()

# Test the connection
if db_connector.connect():
    print("\n🚀 Database connector ready for use!")
else:
    print("\n❌ Database connection failed. Please check your configuration.")

✅ Successfully connected to MSSQL database!
📊 Server: localhost:1434
🗄️ Database: AdventureWorks2019

🚀 Database connector ready for use!


## 3. Extract Database Metadata and Save to Excel

This section extracts comprehensive metadata from the MSSQL database including tables, columns, data types, and descriptions, then saves it to an Excel file for use in LLM prompts.

In [3]:
class DatabaseMetadataExtractor:
    """
    Extract comprehensive metadata from MSSQL database
    """
    
    def __init__(self, db_connector):
        self.db_connector = db_connector
        self.metadata_df = None
        
    def extract_full_metadata(self):
        """Extract comprehensive metadata including extended properties"""
        query = """
        SELECT 
            t.TABLE_SCHEMA,
            t.TABLE_NAME,
            c.COLUMN_NAME,
            c.DATA_TYPE,
            c.CHARACTER_MAXIMUM_LENGTH,
            c.NUMERIC_PRECISION,
            c.NUMERIC_SCALE,
            c.IS_NULLABLE,
            c.COLUMN_DEFAULT,
            c.ORDINAL_POSITION,
            -- Try to get column descriptions from extended properties
            ISNULL(ep.value, '') as COLUMN_DESCRIPTION,
            -- Additional table information
            t.TABLE_TYPE,
            -- Create a readable data type description
            CASE 
                WHEN c.DATA_TYPE IN ('varchar', 'char', 'nvarchar', 'nchar') 
                    THEN c.DATA_TYPE + '(' + CAST(c.CHARACTER_MAXIMUM_LENGTH as varchar(10)) + ')'
                WHEN c.DATA_TYPE IN ('decimal', 'numeric') 
                    THEN c.DATA_TYPE + '(' + CAST(c.NUMERIC_PRECISION as varchar(10)) + ',' + CAST(c.NUMERIC_SCALE as varchar(10)) + ')'
                ELSE c.DATA_TYPE
            END as FULL_DATA_TYPE
        FROM INFORMATION_SCHEMA.TABLES t
        INNER JOIN INFORMATION_SCHEMA.COLUMNS c 
            ON t.TABLE_SCHEMA = c.TABLE_SCHEMA 
            AND t.TABLE_NAME = c.TABLE_NAME
        LEFT JOIN sys.tables st 
            ON st.name = t.TABLE_NAME 
            AND st.schema_id = SCHEMA_ID(t.TABLE_SCHEMA)
        LEFT JOIN sys.columns sc 
            ON sc.object_id = st.object_id 
            AND sc.name = c.COLUMN_NAME
        LEFT JOIN sys.extended_properties ep 
            ON ep.major_id = sc.object_id 
            AND ep.minor_id = sc.column_id 
            AND ep.name = 'MS_Description'
        WHERE t.TABLE_TYPE = 'BASE TABLE'
        ORDER BY t.TABLE_SCHEMA, t.TABLE_NAME, c.ORDINAL_POSITION
        """
        
        print("🔍 Extracting database metadata...")
        self.metadata_df = self.db_connector.execute_query(query)
        
        if self.metadata_df is not None:
            print(f"✅ Successfully extracted metadata for {len(self.metadata_df)} columns")
            print(f"📊 Found {self.metadata_df['TABLE_NAME'].nunique()} tables")
            print(f"🏷️ Schemas: {', '.join(self.metadata_df['TABLE_SCHEMA'].unique())}")
            
            # Add inferred descriptions for columns without descriptions
            self.metadata_df['INFERRED_DESCRIPTION'] = self.metadata_df.apply(
                self._infer_column_description, axis=1
            )
            
            return self.metadata_df
        else:
            print("❌ Failed to extract metadata")
            return None
    
    def _infer_column_description(self, row):
        """Infer description based on column name and data type"""
        column_name = row['COLUMN_NAME'].lower()
        data_type = row['DATA_TYPE'].lower()
        
        # Use existing description if available
        if row['COLUMN_DESCRIPTION'] and row['COLUMN_DESCRIPTION'].strip():
            return row['COLUMN_DESCRIPTION']
        
        # Common patterns for column descriptions
        if 'id' in column_name:
            if column_name.endswith('_id') or column_name == 'id':
                return "Unique identifier"
            else:
                return f"Identifier for {column_name.replace('_id', '').replace('id', '')}"
        
        if 'name' in column_name:
            return f"Name of the {column_name.replace('_name', '').replace('name', '')}"
        
        if 'date' in column_name or 'time' in column_name:
            return f"Date/time value for {column_name}"
        
        if 'email' in column_name:
            return "Email address"
        
        if 'phone' in column_name:
            return "Phone number"
        
        if 'address' in column_name:
            return "Address information"
        
        if 'status' in column_name:
            return "Status indicator"
        
        if 'flag' in column_name:
            return "Boolean flag"
        
        if 'count' in column_name or 'qty' in column_name:
            return "Quantity or count value"
        
        if 'amount' in column_name or 'price' in column_name:
            return "Monetary amount"
        
        # Default based on data type
        if data_type in ['varchar', 'nvarchar', 'char', 'nchar', 'text']:
            return f"Text field: {column_name}"
        elif data_type in ['int', 'bigint', 'smallint', 'tinyint']:
            return f"Integer value: {column_name}"
        elif data_type in ['decimal', 'numeric', 'float', 'real', 'money']:
            return f"Numeric value: {column_name}"
        elif data_type in ['datetime', 'datetime2', 'date', 'time']:
            return f"Date/time value: {column_name}"
        elif data_type in ['bit']:
            return f"Boolean value: {column_name}"
        else:
            return f"Data field: {column_name}"
    
    def save_to_excel(self, filename="database_metadata.xlsx"):
        """Save metadata to Excel file"""
        if self.metadata_df is None:
            print("❌ No metadata to save. Run extract_full_metadata() first.")
            return False
        
        try:
            filepath = os.path.join(os.getcwd(), filename)
            
            with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
                # Main metadata sheet
                self.metadata_df.to_excel(writer, sheet_name='Metadata', index=False)
                
                # Summary sheet
                summary_data = {
                    'Metric': [
                        'Total Tables',
                        'Total Columns', 
                        'Schemas',
                        'Data Types Used',
                        'Tables with Descriptions',
                        'Columns with Descriptions'
                    ],
                    'Value': [
                        self.metadata_df['TABLE_NAME'].nunique(),
                        len(self.metadata_df),
                        ', '.join(self.metadata_df['TABLE_SCHEMA'].unique()),
                        ', '.join(self.metadata_df['DATA_TYPE'].unique()),
                        len(self.metadata_df[self.metadata_df['COLUMN_DESCRIPTION'].notna()]['TABLE_NAME'].unique()),
                        len(self.metadata_df[self.metadata_df['COLUMN_DESCRIPTION'].notna()])
                    ]
                }
                
                summary_df = pd.DataFrame(summary_data)
                summary_df.to_excel(writer, sheet_name='Summary', index=False)
                
                # Table list sheet
                table_summary = self.metadata_df.groupby(['TABLE_SCHEMA', 'TABLE_NAME']).agg({
                    'COLUMN_NAME': 'count',
                    'COLUMN_DESCRIPTION': lambda x: x.notna().sum()
                }).rename(columns={
                    'COLUMN_NAME': 'Column_Count',
                    'COLUMN_DESCRIPTION': 'Described_Columns'
                }).reset_index()
                
                table_summary.to_excel(writer, sheet_name='Tables', index=False)
            
            print(f"✅ Metadata saved to: {filepath}")
            print(f"📊 File size: {os.path.getsize(filepath)} bytes")
            return True
            
        except Exception as e:
            print(f"❌ Error saving to Excel: {str(e)}")
            return False

# Initialize metadata extractor
metadata_extractor = DatabaseMetadataExtractor(db_connector)

# Extract metadata if database connection is available
if db_connector.engine:
    metadata_df = metadata_extractor.extract_full_metadata()
    
    if metadata_df is not None:
        print("\n📋 Sample metadata:")
        display(metadata_df.head(10))
        
        # Save to Excel
        if metadata_extractor.save_to_excel(f"metadata_{db_connector.database}.xlsx"):
            print("\n💾 Metadata successfully saved to Excel file!")
    else:
        print("\n❌ Failed to extract metadata")
else:
    print("\n⚠️ No database connection available. Skipping metadata extraction.")

🔍 Extracting database metadata...
✅ Successfully extracted metadata for 648 columns
📊 Found 71 tables
🏷️ Schemas: dbo, HumanResources, Person, Production, Purchasing, Sales

📋 Sample metadata:
✅ Successfully extracted metadata for 648 columns
📊 Found 71 tables
🏷️ Schemas: dbo, HumanResources, Person, Production, Purchasing, Sales

📋 Sample metadata:


,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,NUMERIC_PRECISION,NUMERIC_SCALE,IS_NULLABLE,COLUMN_DEFAULT,ORDINAL_POSITION,COLUMN_DESCRIPTION,TABLE_TYPE,FULL_DATA_TYPE,INFERRED_DESCRIPTION
0,dbo,AWBuildVersion,SystemInformationID,tinyint,NaN,3.0,0.0,NO,None,1,b'Primary key for AWBuildVersion records.',BASE TABLE,tinyint,b'Primary key for AWBuildVersion records.'
1,dbo,AWBuildVersion,SystemInformationID,tinyint,NaN,3.0,0.0,NO,None,1,b'Clustered index created by a primary key con...,BASE TABLE,tinyint,b'Clustered index created by a primary key con...
2,dbo,AWBuildVersion,Database Version,nvarchar,25.0,NaN,NaN,NO,None,2,b'Version number of the database in 9.yy.mm.dd...,BASE TABLE,nvarchar(25),b'Version number of the database in 9.yy.mm.dd...
3,dbo,AWBuildVersion,VersionDate,datetime,NaN,NaN,NaN,NO,None,3,b'Date and time the record was last updated.',BASE TABLE,datetime,b'Date and time the record was last updated.'
4,dbo,AWBuildVersion,ModifiedDate,datetime,NaN,NaN,NaN,NO,(getdate()),4,b'Date and time the record was last updated.',BASE TABLE,datetime,b'Date and time the record was last updated.'
5,dbo,DatabaseLog,DatabaseLogID,int,NaN,10.0,0.0,NO,None,1,b'Primary key for DatabaseLog records.',BASE TABLE,int,b'Primary key for DatabaseLog records.'
6,dbo,DatabaseLog,PostTime,datetime,NaN,NaN,NaN,NO,None,2,b'The date and time the DDL change occurred.',BASE TABLE,datetime,b'The date and time the DDL change occurred.'
7,dbo,DatabaseLog,PostTime,datetime,NaN,NaN,NaN,NO,None,2,b'Nonclustered index created by a primary key ...,BASE TABLE,datetime,b'Nonclustered index created by a primary key ...
8,dbo,DatabaseLog,DatabaseUser,nvarchar,128.0,NaN,NaN,NO,None,3,b'The user who implemented the DDL change.',BASE TABLE,nvarchar(128),b'The user who implemented the DDL change.'
9,dbo,DatabaseLog,Event,nvarchar,128.0,NaN,NaN,NO,None,4,b'The type of DDL statement that was executed.',BASE TABLE,nvarchar(128),b'The type of DDL statement that was executed.'


✅ Metadata saved to: /home/ubuntu/git-projects/personal/github.com/elasticsearch_opensearch/opensearch/my_tutorial/scripts/5. REALTIME_PROJECTS/4. text_to_bi/metadata_AdventureWorks2019.xlsx
📊 File size: 53214 bytes

💾 Metadata successfully saved to Excel file!


## 4. Load Metadata into DataFrame

Load the saved Excel metadata file into a pandas DataFrame for use in LLM prompts.

In [4]:
def load_metadata_from_excel(filename=f"metadata_{db_connector.database}.xlsx"):
    """
    Load metadata from Excel file into DataFrame
    """
    try:
        filepath = os.path.join(os.getcwd(), filename)
        
        if not os.path.exists(filepath):
            print(f"❌ Metadata file not found: {filepath}")
            print("💡 Run the metadata extraction section first to create the file.")
            return None
        
        # Load the main metadata sheet
        df_meta = pd.read_excel(filepath, sheet_name='Metadata')
        
        print(f"✅ Successfully loaded metadata from: {filepath}")
        print(f"📊 Loaded {len(df_meta)} rows of metadata")
        print(f"🏷️ Tables: {df_meta['TABLE_NAME'].nunique()}")
        print(f"📋 Schemas: {', '.join(df_meta['TABLE_SCHEMA'].unique())}")
        
        # Display sample data
        print("\n📋 Sample metadata structure:")
        display(df_meta.head())
        
        # Show data types distribution
        print(f"\n📊 Data types in database:")
        data_type_counts = df_meta['DATA_TYPE'].value_counts()
        for dtype, count in data_type_counts.head(10).items():
            print(f"  {dtype}: {count} columns")
        
        return df_meta
        
    except Exception as e:
        print(f"❌ Error loading metadata: {str(e)}")
        return None

# Load metadata into df_meta variable
df_meta = load_metadata_from_excel("mssql_metadata.xlsx")

# If loading from file fails, try to use the extracted metadata
if df_meta is None and 'metadata_df' in locals() and metadata_df is not None:
    print("\n🔄 Using recently extracted metadata instead...")
    df_meta = metadata_df.copy()
    print(f"✅ Using in-memory metadata: {len(df_meta)} rows")

if df_meta is not None:
    # Prepare metadata for LLM prompts by creating a simplified format
    # Create a table-focused view for better LLM understanding
    df_meta['TABLE_FULL_NAME'] = df_meta['TABLE_SCHEMA'] + '.' + df_meta['TABLE_NAME']
    df_meta['COLUMN_INFO'] = (
        df_meta['COLUMN_NAME'] + ' (' + 
        df_meta['FULL_DATA_TYPE'] + 
        ', ' + df_meta['IS_NULLABLE'].map({'YES': 'nullable', 'NO': 'not null'}) + ')'
    )
    
    print(f"\n🚀 Metadata ready for LLM queries!")
    print(f"📝 Use 'df_meta' variable for text-to-SQL generation")
else:
    print("\n❌ No metadata available. Please check database connection and run metadata extraction.")

❌ Metadata file not found: /home/ubuntu/git-projects/personal/github.com/elasticsearch_opensearch/opensearch/my_tutorial/scripts/5. REALTIME_PROJECTS/4. text_to_bi/mssql_metadata.xlsx
💡 Run the metadata extraction section first to create the file.

🔄 Using recently extracted metadata instead...
✅ Using in-memory metadata: 648 rows

🚀 Metadata ready for LLM queries!
📝 Use 'df_meta' variable for text-to-SQL generation


## 5. Text-to-SQL with LLM (Universal Provider Support)

This section implements universal LLM integration supporting multiple providers:
- **OpenAI** (GPT-4o, GPT-4o-mini, GPT-4-turbo, GPT-3.5-turbo)
- **Anthropic** (Claude-3.5-Sonnet, Claude-3.5-Haiku, Claude-3-Opus)  
- **Google** (Gemini-1.5-Pro, Gemini-1.5-Flash, Gemini-1.0-Pro)
- **DeepSeek** (DeepSeek-Chat, DeepSeek-Coder)

In [5]:
class UniversalLLMProvider:
    """
    Universal LLM provider supporting multiple APIs
    """
    
    def __init__(self):
        self.providers_config = {
            "openai": {
                "env_var": "OPENAI_API_KEY",
                "models": ["gpt-4o", "gpt-4o-mini", "gpt-4-turbo", "gpt-3.5-turbo"],
                "default_model": "gpt-4o-mini"
            },
            "anthropic": {
                "env_var": "ANTHROPIC_API_KEY",
                "models": ["claude-3-5-sonnet-20241022", "claude-3-5-haiku-20241022", "claude-3-opus-20240229"],
                "default_model": "claude-3-5-sonnet-20241022"
            },
            "google": {
                "env_var": "GOOGLE_API_KEY", 
                "models": ["gemini-1.5-pro", "gemini-1.5-flash", "gemini-1.0-pro"],
                "default_model": "gemini-1.5-flash"
            },
            "deepseek": {
                "env_var": "DEEPSEEK_API_KEY",
                "models": ["deepseek-chat", "deepseek-coder"],
                "default_model": "deepseek-chat"
            }
        }
    
    def query_openai(self, prompt, api_key, model="gpt-4o-mini"):
        """Query OpenAI GPT models"""
        url = "https://api.openai.com/v1/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        
        data = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
            "max_tokens": 2000
        }
        
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content']
    
    def query_anthropic(self, prompt, api_key, model="claude-3-5-sonnet-20241022"):
        """Query Anthropic Claude models"""
        url = "https://api.anthropic.com/v1/messages"
        headers = {
            "Content-Type": "application/json",
            "x-api-key": api_key,
            "anthropic-version": "2023-06-01"
        }
        
        data = {
            "model": model,
            "max_tokens": 2000,
            "temperature": 0.1,
            "messages": [{"role": "user", "content": prompt}]
        }
        
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        return result['content'][0]['text']
    
    def query_google_gemini(self, prompt, api_key, model="gemini-1.5-flash"):
        """Query Google Gemini models"""
        url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:generateContent?key={api_key}"
        headers = {"Content-Type": "application/json"}
        
        data = {
            "contents": [{"parts": [{"text": prompt}]}],
            "generationConfig": {
                "temperature": 0.1,
                "maxOutputTokens": 2000
            }
        }
        
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        return result['candidates'][0]['content']['parts'][0]['text']
    
    def query_deepseek(self, prompt, api_key, model="deepseek-chat"):
        """Query DeepSeek models"""
        url = "https://api.deepseek.com/v1/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        
        data = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1
        }
        
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content']
    
    def query_llm(self, prompt, provider, model=None):
        """Universal method to query any LLM provider"""
        if provider not in self.providers_config:
            raise ValueError(f"Unsupported provider: {provider}")
        
        config = self.providers_config[provider]
        api_key = os.getenv(config["env_var"])
        
        if not api_key:
            raise ValueError(f"API key not found for {provider}. Set {config['env_var']} environment variable.")
        
        selected_model = model or config["default_model"]
        
        try:
            if provider == "openai":
                return self.query_openai(prompt, api_key, selected_model)
            elif provider == "anthropic":
                return self.query_anthropic(prompt, api_key, selected_model)
            elif provider == "google":
                return self.query_google_gemini(prompt, api_key, selected_model)
            elif provider == "deepseek":
                return self.query_deepseek(prompt, api_key, selected_model)
        except Exception as e:
            raise Exception(f"Error querying {provider}: {str(e)}")

class SQLGenerator:
    """
    Generate SQL queries from natural language using LLMs
    """
    
    def __init__(self, metadata_df, llm_provider):
        self.metadata_df = metadata_df
        self.llm_provider = llm_provider
    
    def analyze_query_requirements(self, user_query, provider, model=None):
        """Analyze user query to identify relevant tables and columns"""
        if self.metadata_df is None:
            raise ValueError("No metadata available. Load metadata first.")
        
        # Prepare metadata context for LLM
        metadata_context = self._prepare_metadata_context()
        
        prompt = f"""
        Given the following database metadata for MSSQL server database and user query, identify the most relevant tables and columns:

        DATABASE METADATA:
        {metadata_context}

        USER QUERY: {user_query}

        Please respond with:
        1. Relevant tables: List table names (include schema if needed)
        2. Relevant columns from those tables  
        3. Brief explanation of why these are relevant
        4. Any join conditions that might be needed

        Format your response as structured text.
        """
        
        response = self.llm_provider.query_llm(prompt, provider, model)
        return response
    
    def generate_sql_query(self, user_query, relevant_tables_columns, provider, model=None):
        """Generate SQL query based on analysis"""
        prompt = f"""
        Based on the following information, generate a Microsoft SQL Server query to answer the user's question:

        USER QUERY: {user_query}

        RELEVANT TABLES AND COLUMNS:
        {relevant_tables_columns}

        Please generate a well-formatted SQL Server query that:
        1. Uses proper Microsoft SQL Server compatible syntax
        2. Includes appropriate JOINs if multiple tables are needed
        3. Uses proper WHERE clauses for filtering
        4. Uses appropriate aggregate functions if needed
        5. Is optimized for performance
        6. Includes proper schema names (schema.table format)
        7. Uses TOP instead of LIMIT for SQL Server
        8. Self-check for accurate syntax for partition, Ranking and other advanced sql functions for e.g. partition should have over clause
        9. Self-check that every column name, table and schema produced in final SQL exists in the database metadata provided. Check for exact spelling

        Return ONLY the SQL query without any additional explanation or markdown formatting.
        """
        
        response = self.llm_provider.query_llm(prompt, provider, model)
        
        # Clean up the response to extract just the SQL
        sql_query = self._clean_sql_response(response)
        return sql_query
    
    def _prepare_metadata_context(self, max_tables=50):
        """Prepare metadata context for LLM prompts"""
        if self.metadata_df is None:
            return "No metadata available"
        
        # Group by table to create table-centric view
        table_info = []
        
        for table_name, group in self.metadata_df.groupby(['TABLE_SCHEMA', 'TABLE_NAME']):
            schema, table = table_name
            columns_info = []
            
            for _, row in group.iterrows():
                col_desc = row.get('INFERRED_DESCRIPTION', '')
                col_info = f"  {row['COLUMN_NAME']} ({row['FULL_DATA_TYPE']}) - {col_desc}"
                columns_info.append(col_info)
            
            table_desc = f"Table: {schema}.{table}\n" + "\n".join(columns_info[:20])  # Limit columns per table
            table_info.append(table_desc)
            
            if len(table_info) >= max_tables:
                break
        
        return "\n\n".join(table_info)
    
    def _clean_sql_response(self, response):
        """Clean LLM response to extract SQL query"""
        # Remove markdown code blocks
        import re
        response = re.sub(r'```sql\s*', '', response, flags=re.IGNORECASE)
        response = re.sub(r'```\s*', '', response)
        
        # Remove common prefixes
        response = re.sub(r'^(sql\s*:?\s*)', '', response, flags=re.IGNORECASE | re.MULTILINE)
        
        # Clean up whitespace
        response = response.strip()
        
        return response
    
    def process_nl_query(self, user_query, provider, model=None):
        """Complete process: analyze query -> generate SQL"""
        print(f"🤖 Processing with {provider.title()}")
        print("=" * 50)
        
        # Step 1: Analyze requirements
        print("Step 1: Analyzing query requirements...")
        analysis = self.analyze_query_requirements(user_query, provider, model)
        print("Analysis:")
        print(analysis)
        print("\n" + "=" * 50)
        
        # Step 2: Generate SQL
        print("Step 2: Generating SQL query...")
        sql_query = self.generate_sql_query(user_query, analysis, provider, model)
        print("Generated SQL:")
        print(sql_query)
        
        return analysis, sql_query

# Initialize the LLM provider and SQL generator
llm_provider = UniversalLLMProvider()

if df_meta is not None:
    sql_generator = SQLGenerator(df_meta, llm_provider)
    print("🚀 SQL Generator ready!")
    print(f"📊 Loaded metadata for {df_meta['TABLE_NAME'].nunique()} tables")
else:
    print("❌ Cannot initialize SQL Generator - no metadata available")

🚀 SQL Generator ready!
📊 Loaded metadata for 71 tables


## 6. Direct SQL Execution on MSSQL Database

Execute generated SQL queries directly on the MSSQL database and return results as pandas DataFrames.

In [6]:
class SQLExecutor:
    """
    Execute SQL queries on MSSQL database with safety checks and monitoring
    """
    
    def __init__(self, db_connector):
        self.db_connector = db_connector
        self.execution_history = []
    
    def execute_sql_query(self, sql_query, timeout=30, max_rows=10000):
        """
        Execute SQL query with safety checks and monitoring
        
        Parameters:
        - sql_query: SQL query to execute
        - timeout: Query timeout in seconds
        - max_rows: Maximum rows to return (safety limit)
        """
        if not self.db_connector.engine:
            raise Exception("No database connection available")
        
        # Basic safety checks
        if not self._is_safe_query(sql_query):
            raise Exception("Query contains potentially unsafe operations")
        
        start_time = datetime.now()
        
        try:
            print(f"🔄 Executing SQL query...")
            print("-" * 50)
            print(sql_query)
            print("-" * 50)
            
            # Execute query with timeout
            with self.db_connector.engine.connect() as connection:
                # Set query timeout
                connection = connection.execution_options(autocommit=True)
                
                result_df = pd.read_sql_query(
                    text(sql_query), 
                    connection,
                    chunksize=None
                )
                
                # Apply row limit for safety
                if len(result_df) > max_rows:
                    print(f"⚠️  Result truncated to {max_rows} rows (original: {len(result_df)} rows)")
                    result_df = result_df.head(max_rows)
                
                execution_time = (datetime.now() - start_time).total_seconds()
                
                print(f"✅ Query executed successfully!")
                print(f"⏱️  Execution time: {execution_time:.2f} seconds")
                print(f"📊 Rows returned: {len(result_df)}")
                print(f"📋 Columns: {len(result_df.columns)}")
                
                # Log execution
                self._log_execution(sql_query, len(result_df), execution_time, True, None)
                
                return result_df
                
        except Exception as e:
            execution_time = (datetime.now() - start_time).total_seconds()
            error_msg = str(e)
            
            print(f"❌ Query execution failed!")
            print(f"⏱️  Time before failure: {execution_time:.2f} seconds")
            print(f"🔍 Error: {error_msg}")
            
            # Log failed execution
            self._log_execution(sql_query, 0, execution_time, False, error_msg)
            
            # Provide helpful error suggestions
            self._suggest_error_fixes(error_msg)
            
            return None
    
    def _is_safe_query(self, sql_query):
        """Basic safety checks for SQL queries"""
        sql_upper = sql_query.upper().strip()
        
        # Check for dangerous operations
        dangerous_keywords = [
            'DROP', 'DELETE', 'TRUNCATE', 'ALTER', 'CREATE', 'INSERT', 'UPDATE',
            'EXEC', 'EXECUTE', 'SP_', 'XP_', 'BULK', 'OPENROWSET', 'OPENDATASOURCE'
        ]
        
        for keyword in dangerous_keywords:
            if keyword in sql_upper:
                print(f"❌ Unsafe operation detected: {keyword}")
                return False
        
        # Must start with SELECT (allow WITH for CTEs)
        if not (sql_upper.startswith('SELECT') or sql_upper.startswith('WITH')):
            print(f"❌ Only SELECT queries are allowed")
            return False
        
        return True
    
    def _suggest_error_fixes(self, error_msg):
        """Suggest fixes for common SQL errors"""
        error_lower = error_msg.lower()
        
        suggestions = []
        
        if "invalid object name" in error_lower:
            suggestions.append("• Check table/column names and ensure they exist")
            suggestions.append("• Verify schema names are correct (use schema.table format)")
            suggestions.append("• Check for typos in table or column names")
        
        if "syntax error" in error_lower or "incorrect syntax" in error_lower:
            suggestions.append("• Check SQL syntax - ensure proper SQL Server T-SQL format")
            suggestions.append("• Use TOP instead of LIMIT for SQL Server")
            suggestions.append("• Check for missing commas, parentheses, or quotes")
        
        if "permission" in error_lower or "access" in error_lower:
            suggestions.append("• Check database permissions for your account")
            suggestions.append("• Verify you have SELECT permissions on the tables")
        
        if "timeout" in error_lower:
            suggestions.append("• Query took too long - try adding WHERE clauses to limit data")
            suggestions.append("• Consider using TOP to limit results")
            suggestions.append("• Add appropriate indexes if you have admin access")
        
        if suggestions:
            print("\n💡 Suggestions to fix the error:")
            for suggestion in suggestions:
                print(suggestion)
    
    def _log_execution(self, query, rows_returned, execution_time, success, error_msg):
        """Log query execution for analysis"""
        log_entry = {
            'timestamp': datetime.now(),
            'query': query[:200] + "..." if len(query) > 200 else query,
            'rows_returned': rows_returned,
            'execution_time': execution_time,
            'success': success,
            'error_msg': error_msg
        }
        self.execution_history.append(log_entry)
    
    def get_execution_history(self):
        """Get execution history as DataFrame"""
        if not self.execution_history:
            return pd.DataFrame()
        return pd.DataFrame(self.execution_history)
    
    def get_query_statistics(self):
        """Get execution statistics"""
        if not self.execution_history:
            print("No execution history available")
            return
        
        df_history = self.get_execution_history()
        
        print("📊 Query Execution Statistics:")
        print(f"  Total queries executed: {len(df_history)}")
        print(f"  Successful queries: {df_history['success'].sum()}")
        print(f"  Failed queries: {(~df_history['success']).sum()}")
        print(f"  Average execution time: {df_history['execution_time'].mean():.2f} seconds")
        print(f"  Total rows returned: {df_history['rows_returned'].sum()}")

# Initialize SQL executor
if db_connector.engine:
    sql_executor = SQLExecutor(db_connector)
    print("🚀 SQL Executor ready!")
    print("⚡ Ready to execute queries on MSSQL database")
else:
    print("❌ Cannot initialize SQL Executor - no database connection")

🚀 SQL Executor ready!
⚡ Ready to execute queries on MSSQL database


## 7. Interactive Widgets for NL Query to SQL and Execution

Complete interactive interface for natural language to SQL conversion and execution on MSSQL database.

In [7]:
class DataVisualizationAndInsights:
    """
    Generate visualizations and business insights from DataFrame results
    """
    
    def __init__(self, llm_provider):
        self.llm_provider = llm_provider
        
    def generate_insights(self, df, original_query, provider="openai", model=None):
        """Generate business insights using LLM"""
        if df is None or len(df) == 0:
            return "No data available for insights generation."
        
        # Prepare data summary for LLM
        data_summary = self._prepare_data_summary(df)
        
        prompt = f"""
        Based on the following SQL query results and data summary, provide business insights and recommendations:

        ORIGINAL QUERY: {original_query}

        DATA SUMMARY:
        {data_summary}

        Please provide:
        1. Key findings from the data
        2. Business insights and trends
        3. Potential recommendations or actions
        4. Any anomalies or interesting patterns
        5. Suggested follow-up questions for deeper analysis

        Format your response in clear, business-friendly language.
        """
        
        try:
            insights = self.llm_provider.query_llm(prompt, provider, model)
            return insights
        except Exception as e:
            return f"Error generating insights: {str(e)}"
    
    def _prepare_data_summary(self, df):
        """Prepare a comprehensive summary of the DataFrame"""
        summary = []
        
        # Basic info
        summary.append(f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns")
        summary.append(f"Columns: {', '.join(df.columns.tolist())}")
        
        # Data types and basic stats
        summary.append("\nColumn Information:")
        for col in df.columns:
            dtype = df[col].dtype
            non_null = df[col].notna().sum()
            null_count = df[col].isna().sum()
            
            col_info = f"  {col}: {dtype}, {non_null} non-null, {null_count} null"
            
            # Add statistics for numeric columns
            if pd.api.types.is_numeric_dtype(df[col]):
                try:
                    stats = df[col].describe()
                    col_info += f" (min: {stats['min']:.2f}, max: {stats['max']:.2f}, mean: {stats['mean']:.2f})"
                except:
                    pass
            
            # Add info for categorical columns
            elif pd.api.types.is_object_dtype(df[col]):
                unique_count = df[col].nunique()
                col_info += f" ({unique_count} unique values)"
                if unique_count <= 10:
                    top_values = df[col].value_counts().head(5)
                    col_info += f", top values: {dict(top_values)}"
            
            summary.append(col_info)
        
        # Sample data
        summary.append(f"\nSample Data (first 3 rows):")
        sample_data = df.head(3).to_string()
        summary.append(sample_data)
        
        return "\n".join(summary)
    
    def create_visualizations(self, df, original_query=""):
        """Create appropriate visualizations based on DataFrame content"""
        if df is None or len(df) == 0:
            print("No data available for visualization.")
            return
        
        print("📊 Generating visualizations...")
        
        # Determine appropriate visualizations based on data types
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
        datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
        
        visualizations_created = 0
        
        # 1. Distribution plots for numeric columns
        if numeric_cols:
            self._create_distribution_plots(df, numeric_cols[:4])  # Limit to 4 columns
            visualizations_created += 1
        
        # 2. Bar charts for categorical data
        if categorical_cols:
            self._create_categorical_plots(df, categorical_cols[:3])  # Limit to 3 columns
            visualizations_created += 1
        
        # 3. Correlation heatmap if multiple numeric columns
        if len(numeric_cols) > 1:
            self._create_correlation_heatmap(df, numeric_cols)
            visualizations_created += 1
        
        # 4. Time series if datetime columns exist
        if datetime_cols and numeric_cols:
            self._create_time_series_plots(df, datetime_cols[0], numeric_cols[:2])
            visualizations_created += 1
        
        # 5. Top N analysis for categorical + numeric combinations
        if categorical_cols and numeric_cols:
            self._create_top_n_analysis(df, categorical_cols[0], numeric_cols[0])
            visualizations_created += 1
        
        # 6. Interactive plotly visualization
        self._create_interactive_plot(df, numeric_cols, categorical_cols)
        visualizations_created += 1
        
        if visualizations_created == 0:
            print("⚠️ No suitable visualizations could be created for this data.")
        else:
            print(f"✅ Created {visualizations_created} visualizations")
    
    def _create_distribution_plots(self, df, numeric_cols):
        """Create distribution plots for numeric columns"""
        n_cols = min(len(numeric_cols), 2)
        n_rows = (len(numeric_cols) + 1) // 2
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4*n_rows))
        fig.suptitle('📊 Distribution Analysis', fontsize=16, fontweight='bold')
        
        if n_rows == 1 and n_cols == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = axes
        else:
            axes = axes.flatten()
        
        for i, col in enumerate(numeric_cols[:4]):
            ax = axes[i] if len(axes) > 1 else axes[0]
            
            # Histogram with KDE
            sns.histplot(data=df, x=col, kde=True, ax=ax)
            ax.set_title(f'Distribution of {col}')
            ax.set_xlabel(col)
            ax.set_ylabel('Frequency')
        
        # Hide empty subplots
        for i in range(len(numeric_cols), len(axes)):
            axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    def _create_categorical_plots(self, df, categorical_cols):
        """Create bar charts for categorical columns"""
        n_cols = min(len(categorical_cols), 3)
        
        fig, axes = plt.subplots(1, n_cols, figsize=(6*n_cols, 6))
        fig.suptitle('📈 Categorical Analysis', fontsize=16, fontweight='bold')
        
        if n_cols == 1:
            axes = [axes]
        
        for i, col in enumerate(categorical_cols[:3]):
            ax = axes[i] if n_cols > 1 else axes[0]
            
            # Get top 10 categories to avoid overcrowding
            top_categories = df[col].value_counts().head(10)
            
            # Bar plot
            sns.barplot(x=top_categories.values, y=top_categories.index, ax=ax)
            ax.set_title(f'Top Categories: {col}')
            ax.set_xlabel('Count')
            ax.set_ylabel(col)
        
        plt.tight_layout()
        plt.show()
    
    def _create_correlation_heatmap(self, df, numeric_cols):
        """Create correlation heatmap for numeric columns"""
        if len(numeric_cols) < 2:
            return
        
        plt.figure(figsize=(10, 8))
        correlation_matrix = df[numeric_cols].corr()
        
        sns.heatmap(correlation_matrix, 
                   annot=True, 
                   cmap='coolwarm', 
                   center=0,
                   square=True,
                   fmt='.2f')
        
        plt.title('🔥 Correlation Heatmap', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    def _create_time_series_plots(self, df, date_col, numeric_cols):
        """Create time series plots"""
        fig, axes = plt.subplots(len(numeric_cols), 1, figsize=(12, 4*len(numeric_cols)))
        fig.suptitle('📅 Time Series Analysis', fontsize=16, fontweight='bold')
        
        if len(numeric_cols) == 1:
            axes = [axes]
        
        for i, col in enumerate(numeric_cols):
            ax = axes[i] if len(numeric_cols) > 1 else axes[0]
            
            # Sort by date and plot
            df_sorted = df.sort_values(date_col)
            ax.plot(df_sorted[date_col], df_sorted[col], marker='o', linewidth=2)
            ax.set_title(f'{col} over Time')
            ax.set_xlabel(date_col)
            ax.set_ylabel(col)
            ax.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    def _create_top_n_analysis(self, df, categorical_col, numeric_col):
        """Create top N analysis combining categorical and numeric data"""
        plt.figure(figsize=(12, 6))
        
        # Group by categorical column and aggregate numeric column
        grouped = df.groupby(categorical_col)[numeric_col].agg(['sum', 'mean', 'count']).reset_index()
        grouped = grouped.sort_values('sum', ascending=False).head(15)
        
        # Create subplot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle(f'🏆 Top Analysis: {categorical_col} vs {numeric_col}', fontsize=16, fontweight='bold')
        
        # Top by sum
        sns.barplot(data=grouped.head(10), x='sum', y=categorical_col, ax=ax1)
        ax1.set_title(f'Top 10 by Total {numeric_col}')
        ax1.set_xlabel(f'Total {numeric_col}')
        
        # Top by average
        grouped_avg = grouped.sort_values('mean', ascending=False).head(10)
        sns.barplot(data=grouped_avg, x='mean', y=categorical_col, ax=ax2)
        ax2.set_title(f'Top 10 by Average {numeric_col}')
        ax2.set_xlabel(f'Average {numeric_col}')
        
        plt.tight_layout()
        plt.show()
    
    def _create_interactive_plot(self, df, numeric_cols, categorical_cols):
        """Create interactive Plotly visualization"""
        if not numeric_cols:
            return
        
        print("🎯 Interactive Visualization:")
        
        if len(numeric_cols) >= 2:
            # Scatter plot if we have at least 2 numeric columns
            color_col = categorical_cols[0] if categorical_cols else None
            
            fig = px.scatter(df, 
                           x=numeric_cols[0], 
                           y=numeric_cols[1],
                           color=color_col,
                           title=f"Interactive Scatter Plot: {numeric_cols[0]} vs {numeric_cols[1]}",
                           hover_data=df.columns.tolist())
        
        elif categorical_cols:
            # Bar chart if we have categorical and numeric data
            # Group data for better visualization
            grouped = df.groupby(categorical_cols[0])[numeric_cols[0]].sum().reset_index()
            grouped = grouped.sort_values(numeric_cols[0], ascending=False).head(20)
            
            fig = px.bar(grouped, 
                        x=categorical_cols[0], 
                        y=numeric_cols[0],
                        title=f"Interactive Bar Chart: {categorical_cols[0]} vs {numeric_cols[0]}")
            fig.update_xaxes(tickangle=45)
        
        else:
            # Histogram for single numeric column
            fig = px.histogram(df, 
                             x=numeric_cols[0],
                             title=f"Interactive Histogram: {numeric_cols[0]}")
        
        fig.update_layout(height=500)
        fig.show()

class InteractiveTextToSQL:
    """
    Interactive interface for text-to-SQL with MSSQL execution
    """
    
    def __init__(self, sql_generator, sql_executor, llm_provider):
        self.sql_generator = sql_generator
        self.sql_executor = sql_executor
        self.llm_provider = llm_provider
        self.viz_insights = DataVisualizationAndInsights(llm_provider)
        self.last_result_df = None  # Store last query result for visualization
        self.setup_widgets()
    
    def setup_widgets(self):
        """Create all the interactive widgets"""
        
        # Check available providers
        available_providers = []
        providers_info = self.llm_provider.providers_config
        
        for provider, config in providers_info.items():
            if os.getenv(config["env_var"]):
                available_providers.append((provider.title(), provider))
        
        if not available_providers:
            available_providers = [("No API Keys", "none")]
        
        # Provider selection
        self.provider_selector = widgets.Dropdown(
            options=available_providers,
            value=available_providers[0][1],
            description='LLM Provider:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='300px')
        )
        
        # Model selection
        self.model_selector = widgets.Dropdown(
            options=[],
            description='Model:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='300px')
        )
        
        # Natural language query input
        self.nl_query_input = widgets.Textarea(
            value='Show revenue by month for this year.',
            placeholder='Enter your question in natural language...',
            description='Question:',
            layout=widgets.Layout(width='100%', height='120px'),
            style={'description_width': '120px'}
        )
        
        # Generated SQL display
        self.generated_sql_display = widgets.Textarea(
            value='',
            placeholder='Generated SQL query will appear here...',
            description='Generated SQL:',
            layout=widgets.Layout(width='100%', height='200px'),
            style={'description_width': '120px'}
        )
        
        # Buttons
        self.generate_button = widgets.Button(
            description='🧠 Generate SQL',
            button_style='primary',
            tooltip='Generate SQL from natural language',
            layout=widgets.Layout(width='200px')
        )
        
        self.execute_button = widgets.Button(
            description='▶️ Execute SQL',
            button_style='success',
            tooltip='Execute the generated SQL on database',
            layout=widgets.Layout(width='200px')
        )
        
        self.clear_button = widgets.Button(
            description='🗑️ Clear',
            button_style='warning',
            tooltip='Clear all fields',
            layout=widgets.Layout(width='200px')
        )
        
        self.visualize_button = widgets.Button(
            description='📊 Visualize',
            button_style='info',
            tooltip='Create visualizations from results',
            layout=widgets.Layout(width='200px')
        )
        
        self.insights_button = widgets.Button(
            description='💡 Insights',
            button_style='success',
            tooltip='Generate business insights',
            layout=widgets.Layout(width='200px')
        )
        
        # Output areas
        self.generation_output = widgets.Output()
        self.execution_output = widgets.Output()
        self.visualization_output = widgets.Output()
        self.insights_output = widgets.Output()
        
        # Setup event handlers
        self.provider_selector.observe(self.update_model_options, names='value')
        self.generate_button.on_click(self.generate_sql)
        self.execute_button.on_click(self.execute_sql)
        self.clear_button.on_click(self.clear_all)
        self.visualize_button.on_click(self.create_visualizations)
        self.insights_button.on_click(self.generate_insights)
        
        # Initial model update
        self.update_model_options()
    
    def update_model_options(self, change=None):
        """Update model options based on selected provider"""
        provider = self.provider_selector.value
        
        if provider == "none":
            self.model_selector.options = [("No Models", "none")]
            return
        
        if provider in self.llm_provider.providers_config:
            models = self.llm_provider.providers_config[provider]["models"]
            model_options = [(model, model) for model in models]
            self.model_selector.options = model_options
            if model_options:
                self.model_selector.value = model_options[0][1]
    
    def generate_sql(self, button):
        """Generate SQL from natural language query"""
        with self.generation_output:
            clear_output()
            
            nl_query = self.nl_query_input.value.strip()
            provider = self.provider_selector.value
            model = self.model_selector.value
            
            if not nl_query:
                print("❌ Please enter a natural language question")
                return
            
            if provider == "none":
                print("❌ No LLM provider available. Please set API keys.")
                return
            
            try:
                print(f"🤖 Generating SQL using {provider.title()} ({model})")
                print("=" * 60)
                
                # Generate SQL using the SQL generator
                analysis, sql_query = self.sql_generator.process_nl_query(
                    nl_query, provider, model
                )
                
                # Update the SQL display
                self.generated_sql_display.value = sql_query
                
                print("\n✅ SQL generation completed!")
                print("👆 Review the generated SQL above and click 'Execute SQL' to run it.")
                
            except Exception as e:
                print(f"❌ Error generating SQL: {str(e)}")
    
    def execute_sql(self, button):
        """Execute the generated SQL query"""
        with self.execution_output:
            clear_output()
            
            sql_query = self.generated_sql_display.value.strip()
            
            if not sql_query:
                print("❌ No SQL query to execute. Generate one first.")
                return
            
            try:
                # Execute the SQL query
                result_df = self.sql_executor.execute_sql_query(sql_query)
                
                if result_df is not None:
                    # Store the result for visualization and insights
                    self.last_result_df = result_df
                    
                    print(f"\n📊 Query Results ({len(result_df)} rows, {len(result_df.columns)} columns):")
                    
                    # Display results
                    if len(result_df) > 0:
                        display(result_df)
                        
                        # Show column info
                        print(f"\n📋 Column Information:")
                        for col in result_df.columns:
                            dtype = result_df[col].dtype
                            non_null = result_df[col].notna().sum()
                            print(f"  {col}: {dtype} ({non_null}/{len(result_df)} non-null)")
                        
                        print(f"\n🎯 Data ready for visualization and insights!")
                        print("Click 'Visualize' or 'Insights' buttons to explore the data further.")
                    else:
                        print("No rows returned by the query.")
                        self.last_result_df = None
                else:
                    print("❌ Query execution failed. Check the error message above.")
                    self.last_result_df = None
                
            except Exception as e:
                print(f"❌ Execution error: {str(e)}")
                self.last_result_df = None
    
    def create_visualizations(self, button):
        """Create visualizations from the last query result"""
        with self.visualization_output:
            clear_output()
            
            if self.last_result_df is None or len(self.last_result_df) == 0:
                print("❌ No data available for visualization.")
                print("Execute a SQL query first to generate data.")
                return
            
            try:
                print("🎨 Creating visualizations from query results...")
                print("=" * 50)
                
                # Create visualizations
                self.viz_insights.create_visualizations(
                    self.last_result_df, 
                    self.nl_query_input.value
                )
                
                print("\n✅ Visualizations created successfully!")
                
            except Exception as e:
                print(f"❌ Error creating visualizations: {str(e)}")
    
    def generate_insights(self, button):
        """Generate business insights from the last query result"""
        with self.insights_output:
            clear_output()
            
            if self.last_result_df is None or len(self.last_result_df) == 0:
                print("❌ No data available for insights generation.")
                print("Execute a SQL query first to generate data.")
                return
            
            provider = self.provider_selector.value
            model = self.model_selector.value
            
            if provider == "none":
                print("❌ No LLM provider available for insights generation.")
                return
            
            try:
                print(f"🧠 Generating business insights using {provider.title()}...")
                print("=" * 50)
                
                # Generate insights
                insights = self.viz_insights.generate_insights(
                    self.last_result_df,
                    self.nl_query_input.value,
                    provider,
                    model
                )
                
                print("💡 Business Insights & Recommendations:")
                print("-" * 40)
                print(insights)
                
                print("\n✅ Insights generated successfully!")
                
            except Exception as e:
                print(f"❌ Error generating insights: {str(e)}")
    
    def clear_all(self, button):
        """Clear all input and output fields"""
        self.nl_query_input.value = ''
        self.generated_sql_display.value = ''
        self.last_result_df = None
        
        with self.generation_output:
            clear_output()
        
        with self.execution_output:
            clear_output()
            
        with self.visualization_output:
            clear_output()
            
        with self.insights_output:
            clear_output()
        
        print("🗑️ All fields cleared!")
    
    def display_interface(self):
        """Display the complete interface"""
        
        # Status display
        status_html = self.get_status_html()
        
        # Create the layout
        interface = widgets.VBox([
            widgets.HTML("<h2>🚀 MSSQL Text-to-SQL Interface</h2>"),
            widgets.HTML(status_html),
            
            # Provider and model selection
            widgets.HTML("<h4>🤖 LLM Configuration</h4>"),
            widgets.HBox([self.provider_selector, self.model_selector]),
            
            # Query input
            widgets.HTML("<h4>💬 Natural Language Query</h4>"),
            self.nl_query_input,
            
            # Action buttons
            widgets.HTML("<h4>⚡ Actions</h4>"),
            widgets.HBox([
                self.generate_button, 
                self.execute_button, 
                self.visualize_button,
                self.insights_button,
                self.clear_button
            ]),
            
            # Generated SQL
            widgets.HTML("<h4>📝 Generated SQL</h4>"),
            self.generated_sql_display,
            
            # Outputs
            widgets.HTML("<h4>🔍 Generation Analysis</h4>"),
            self.generation_output,
            
            widgets.HTML("<h4>📊 Execution Results</h4>"),
            self.execution_output,
            
            widgets.HTML("<h4>📈 Data Visualizations</h4>"),
            self.visualization_output,
            
            widgets.HTML("<h4>💡 Business Insights</h4>"),
            self.insights_output
        ])
        
        return interface
    
    def get_status_html(self):
        """Get HTML status display"""
        # Check database connection
        db_status = "🟢 Connected" if self.sql_executor.db_connector.engine else "🔴 Disconnected"
        
        # Check metadata
        metadata_status = "🟢 Loaded" if self.sql_generator.metadata_df is not None else "🔴 Not Available"
        
        # Check API keys
        available_providers = []
        for provider, config in self.llm_provider.providers_config.items():
            if os.getenv(config["env_var"]):
                available_providers.append(provider.title())
        
        llm_status = f"🟢 {', '.join(available_providers)}" if available_providers else "🔴 No API Keys"
        
        html = f"""
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin: 10px 0;">
            <b>System Status:</b><br>
            📊 Database: {db_status}<br>
            📋 Metadata: {metadata_status}<br>
            🤖 LLM Providers: {llm_status}
        </div>
        """
        
        return html

# Initialize and display the interface
if 'sql_generator' in locals() and 'sql_executor' in locals():
    interface = InteractiveTextToSQL(sql_generator, sql_executor, llm_provider)
    
    print("🎉 Interactive Text-to-SQL Interface Ready!")
    print("\n💡 Usage Instructions:")
    print("1. Select your preferred LLM provider and model")
    print("2. Enter your question in natural language")
    print("3. Click 'Generate SQL' to create the query")
    print("4. Review the generated SQL")
    print("5. Click 'Execute SQL' to run it on the database")
    print("6. Click 'Visualize' to create charts and graphs")
    print("7. Click 'Insights' to generate business recommendations")
    print("8. View the results, visualizations, and insights below")
    
    print("\n📝 Example Questions:")
    examples = [
        "Show me all customers from California",
        "Find the top 10 products by sales",
        "List employees hired in the last year",
        "What are the most recent orders?",
        "Show revenue by month for all years in the data",
        "Segment customers based on their order values and frequency into high, medium, and low value segments."
    ]
    
    for i, example in enumerate(examples, 1):
        print(f"  {i}. {example}")
    
    print("\n📊 Visualization-Friendly Queries:")
    viz_examples = [
        "Sales trends by month and product category",
        "Customer segmentation by purchase behavior", 
        "Regional performance comparison",
        "Employee productivity metrics by department",
        "Inventory levels and turnover rates"
    ]
    
    for i, example in enumerate(viz_examples, 1):
        print(f"  {i}. {example}")
    
    print("\n" + "="*60)
    
    # Display the interface
    display(interface.display_interface())
    
else:
    print("❌ Cannot create interface - missing required components")
    print("Please ensure database connection and metadata are available")

🎉 Interactive Text-to-SQL Interface Ready!

💡 Usage Instructions:
1. Select your preferred LLM provider and model
2. Enter your question in natural language
3. Click 'Generate SQL' to create the query
4. Review the generated SQL
5. Click 'Execute SQL' to run it on the database
6. Click 'Visualize' to create charts and graphs
7. Click 'Insights' to generate business recommendations
8. View the results, visualizations, and insights below

📝 Example Questions:
  1. Show me all customers from California
  2. Find the top 10 products by sales
  3. List employees hired in the last year
  4. What are the most recent orders?
  5. Show revenue by month for this year
  6. Segment customers based on their order values and frequency into high, medium, and low value segments.

📊 Visualization-Friendly Queries:
  1. Sales trends by month and product category
  2. Customer segmentation by purchase behavior
  3. Regional performance comparison
  4. Employee productivity metrics by department
  5. Inve

## 🎯 Summary and Next Steps

### ✅ What This Notebook Provides

1. **MSSQL Database Integration** - Secure connection to SQL Server using pymssql with comprehensive error handling
2. **Automated Metadata Extraction** - Extracts table/column information and saves to Excel for reuse
3. **Multi-Provider LLM Support** - Works with OpenAI, Anthropic, Google Gemini, and DeepSeek
4. **Intelligent SQL Generation** - Converts natural language to optimized T-SQL queries
5. **Direct Database Execution** - Runs queries safely on MSSQL with monitoring and limits
6. **Interactive Interface** - User-friendly widgets for the complete workflow

### 🔧 Configuration Checklist

Before using this notebook:

- [ ] Set up MSSQL connection environment variables (server, database, username, password, port)
- [ ] Configure at least one LLM API key
- [ ] Install required Python packages (pymssql instead of pyodbc)
- [ ] Test database connectivity
- [ ] Run metadata extraction
- [ ] Verify the interactive interface loads

### 🚀 Usage Workflow

1. **Setup** - Configure environment variables and run initial cells
2. **Connect** - Establish database connection using pymssql
3. **Extract** - Generate and save metadata to Excel
4. **Query** - Use the interactive interface to ask questions in natural language
5. **Execute** - Run generated SQL queries on your database
6. **Analyze** - Review results and refine queries as needed

### 💡 Pro Tips

- **Be Specific**: More detailed questions produce better SQL queries
- **Use Examples**: Reference specific table/column names when known
- **Set Limits**: Always include row limits for large datasets
- **Review SQL**: Check generated queries before execution
- **Save Results**: Export important query results to files

### 🔐 Security Notes

- Only SELECT queries are allowed for safety
- Row limits prevent overwhelming results
- Query timeout prevents long-running operations
- All database operations are logged for monitoring
- SQL Server authentication required (Windows Auth not supported with pymssql)

### 📚 Next Steps

- Extend with custom functions for your specific domain
- Add query result caching for frequently used queries
- Implement query optimization suggestions
- Create saved query templates for common patterns
- Add data visualization capabilities

---

**🎉 You're ready to start querying your MSSQL database with natural language!**

## 🎨 Enhanced Features: Data Visualization & Business Insights

### 📊 Automatic Data Visualization

When you execute a SQL query, the system now provides intelligent visualization capabilities:

#### **Smart Chart Selection**
- **Distribution Plots**: Histograms and KDE plots for numeric data
- **Bar Charts**: Top categories for categorical data
- **Correlation Heatmaps**: Relationships between numeric variables
- **Time Series**: Trends over time for datetime columns
- **Top N Analysis**: Ranking analysis for categorical + numeric combinations
- **Interactive Plots**: Plotly visualizations for exploration

#### **Visualization Types Created**
1. **Statistical Distributions** - Understand data spread and patterns
2. **Categorical Analysis** - See frequency and proportions
3. **Correlation Analysis** - Identify relationships between variables
4. **Time-based Trends** - Track changes over time
5. **Ranking & Comparisons** - Top performers and outliers
6. **Interactive Exploration** - Drill-down capabilities

### 💡 AI-Powered Business Insights

The system uses your selected LLM to generate business insights:

#### **Insight Categories**
- **Key Findings**: Main takeaways from the data
- **Business Trends**: Patterns and movements
- **Recommendations**: Actionable suggestions
- **Anomalies**: Unusual patterns requiring attention
- **Follow-up Questions**: Suggestions for deeper analysis

#### **Insight Generation Process**
1. **Data Analysis**: Statistical summary of results
2. **Pattern Recognition**: Identifies trends and outliers
3. **Business Context**: Applies domain knowledge
4. **Actionable Recommendations**: Provides next steps

### 🔄 Workflow Enhancement

The enhanced workflow now includes:

```
Natural Language Query → SQL Generation → Execution → Visualization → Insights
```

Each step builds upon the previous, providing a complete data analysis pipeline from question to actionable insights.

### 📈 Use Cases

Perfect for:
- **Executive Dashboards**: Quick visual summaries
- **Trend Analysis**: Understanding patterns over time
- **Performance Monitoring**: KPI tracking and comparison
- **Data Exploration**: Discovery of hidden patterns
- **Report Generation**: Automated insights for stakeholders